# Notebook
https://www.kaggle.com/williamroe/bi-lstm-with-crf-for-ner

In [1]:
%pip install tensorflow==2.4.0  # 2.5.0  # 2.2.0

     |████████████████████████████████| 394.7 MB 3.7 kB/s  eta 0:00:01     |███████████████▍                | 189.1 MB 42.9 MB/s eta 0:00:05
     |████████████████████████████████| 132 kB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 323 kB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 17.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 66.0 MB/s eta 0:00:01     |█████████▎                      | 1.7 MB 66.0 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 60.4 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 4.9 MB 63.2 MB/s eta 0:00:01
     |█████████████████████████████

In [2]:
# %pip install tensorflow_addons==0.13.0

In [3]:
# %pip install keras-crf==0.2.0

In [4]:
# %pip install tensorflow-cpu==2.4.0

In [5]:
# %pip install git+https://www.github.com/keras-team/keras-contrib.git
# # %pip install keras-contrib==0.1

In [6]:
# https://github.com/xuxingya/tf2crf
# %pip install tf2crf==0.1.32

In [7]:
# %env SM_FRAMEWORK=tf.keras

In [8]:
# %pip uninstall --yes tf-nightly

In [9]:
%pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [731]:
import numpy as np
import pandas as pd 
# import s3fs

what_corpus = 'polnear'
what_type_files = 'train'
what_type_test_file = 'test'
what_type_val_file = 'dev'

# filepath = 's3://sagemaker-studio-528576943967-ssf9zkrg3os/polnear-conll/prepared/'
filepath = '../' + what_corpus + '-conll/prepared/'

In [732]:
train_filename = what_corpus + '_preprocessed_' + what_type_files + '_noBIO.csv'
df_train = pd.read_csv(filepath + train_filename, sep=";")

In [733]:
df_train[df_train['tag'].isna()]

,Unnamed: 0,filename,sentence_idx,word,tag


In [734]:
print(len(df_train.index))
del df_train['Unnamed: 0']
df_train.head(20)

727161


,filename,sentence_idx,word,tag
0,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,Mark,SOURCE
1,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,Cuban,SOURCE
2,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,:,CUE
3,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,I,CONTENT
4,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,would,CONTENT
5,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,consider,CONTENT
6,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,a,CONTENT
7,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,future,CONTENT
8,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,White,CONTENT
9,politico_2016-05-22_mark-cuban-i-d-consider-a-...,1,House,CONTENT


In [735]:
test_filename = what_corpus + '_preprocessed_' + what_type_test_file + '_noBIO.csv'
df_test = pd.read_csv(filepath + test_filename, sep=";")

In [736]:
print(len(df_test.index))
del df_test['Unnamed: 0']
df_test.head(20)

73370


,filename,sentence_idx,word,tag
0,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Hillary,O
1,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Clinton,O
2,huff-post_2016-11-05_hillary-clinton-drops-int...,1,drops,O
3,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Into,O
4,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Detroit,O
5,huff-post_2016-11-05_hillary-clinton-drops-int...,1,as,O
6,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Democrats,SOURCE
7,huff-post_2016-11-05_hillary-clinton-drops-int...,1,get,CUE
8,huff-post_2016-11-05_hillary-clinton-drops-int...,1,nervous,CUE
9,huff-post_2016-11-05_hillary-clinton-drops-int...,1,about,CUE


In [737]:
df_test[df_test['tag'].isna()]

,filename,sentence_idx,word,tag


In [740]:
val_filename = what_corpus + '_preprocessed_' + what_type_val_file + '_noBIO.csv'
df_val = pd.read_csv(filepath + val_filename, sep=";")

In [741]:
print(len(df_val.index))
del df_val['Unnamed: 0']
df_val.head(20)

78348


,filename,sentence_idx,word,tag
0,west-journal_2016-09-29_gold-star-mom-corners-...,1,Gold,O
1,west-journal_2016-09-29_gold-star-mom-corners-...,1,Star,O
2,west-journal_2016-09-29_gold-star-mom-corners-...,1,mom,O
3,west-journal_2016-09-29_gold-star-mom-corners-...,1,Corners,O
4,west-journal_2016-09-29_gold-star-mom-corners-...,1,Obama,O
5,west-journal_2016-09-29_gold-star-mom-corners-...,1,on,O
6,west-journal_2016-09-29_gold-star-mom-corners-...,1,he,SOURCE
7,west-journal_2016-09-29_gold-star-mom-corners-...,1,refusal,CUE
8,west-journal_2016-09-29_gold-star-mom-corners-...,1,to,CONTENT
9,west-journal_2016-09-29_gold-star-mom-corners-...,1,use,CONTENT


In [742]:
df_val[df_val['tag'].isna()]

,filename,sentence_idx,word,tag


In [743]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
#         agg_func = lambda s: ' '.join(s["word"].values.tolist())
        agg_func = lambda s: [(w, t) for w,t in zip(s["word"].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby(["filename", "sentence_idx"]).apply(agg_func)
#         self.grouped = self.dataset.groupby(["sentence_idx"]).apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [744]:
getter_train = SentenceGetter(df_train)
getter_test = SentenceGetter(df_test)
getter_val = SentenceGetter(df_val)

In [745]:
sentences_train = getter_train.sentences
sentences_test = getter_test.sentences
sentences_val = getter_val.sentences

In [746]:
print(len(sentences_train))
print(len(sentences_train)/2)

30343
15171.5


In [747]:
# sentences_train = sentences_train[0:15171]  # parc: 22352]
# print(len(sentences_train))

In [748]:
print(sentences_train[1])

[('in', 'O'), ('we', 'O'), ('last', 'O'), ('epistle', 'O'), ('from', 'O'), ('the', 'O'), ('bottomless', 'O'), ('pit', 'O'), (',', 'O'), ('we', 'SOURCE'), ('mock', 'CUE'), ('the', 'CONTENT'), ('presidential', 'CONTENT'), ('ambition', 'CONTENT'), ('of', 'CONTENT'), ('three', 'CONTENT'), ('republican', 'CONTENT'), ('hopeful', 'CONTENT'), (':', 'CONTENT'), ('Jeb', 'CONTENT'), ('!', 'CONTENT')]


In [749]:
# print(sentences_train[10])

In [750]:
print(len(sentences_test))

3022


In [751]:
# print(sentences_test[10])

In [752]:
maxlen = max([len(s) for s in sentences_train])
print ('Maximum sequence length train:', maxlen)
maxlen_test = max([len(s) for s in sentences_test])
print ('Maximum sequence length test:', maxlen_test)
maxlen_val = max([len(s) for s in sentences_val])
print ('Maximum sequence length val:', maxlen_val)

Maximum sequence length train: 220
Maximum sequence length test: 94
Maximum sequence length val: 95


In [753]:
# words_train = ['', '[UNK]']
# words_train.extend(list(set(df_train["word"].values)))
# words_train.append("ENDPAD")
# words_train[0:5]

In [754]:
sentences_texts_train = [' '.join([w[0] for w in s]) for s in sentences_train]
sentences_texts_train[1]

'in we last epistle from the bottomless pit , we mock the presidential ambition of three republican hopeful : Jeb !'

In [755]:
sentences_texts_test = [' '.join([w[0] for w in s]) for s in sentences_test]
sentences_texts_test[1]

'Washington -lrb- AFP -rrb- -- the Washington Post on Thursday become the latest US newspaper to emphatically endorse Hillary Clinton for the White House , say it be sway as much by she competence as by the alarming specter of a Donald Trump presidency .'

In [756]:
sentences_texts_val = [' '.join([w[0] for w in s]) for s in sentences_val]
sentences_texts_val[1]

"the New York Post report that Hillary Clinton 's team avoid take she to the emergency room follow she medical scare on Sunday in order to `` keep the detail of she medical treatment under wrap . ''"

https://keras.io/examples/nlp/pretrained_word_embeddings/


In [757]:
from tensorflow.data import Dataset
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=50000, output_sequence_length=maxlen)
# vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
# text_ds = Dataset.from_tensor_slices(df_train["word"].values).batch(128)
text_ds = Dataset.from_tensor_slices(sentences_texts_train).batch(128)
vectorizer.adapt(text_ds)

In [758]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'be', 'to']

In [759]:
# output = vectorizer([["the cat sat on the mat"]])
# output.numpy()[0, :6]

In [760]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [761]:
test = ["the", "be", "get", "on", "the"]
[word_index[w] for w in test]

[2, 3, 71, 15, 2]

In [762]:
from gensim.models import KeyedVectors
# word_vecs = KeyedVectors.load_word2vec_format("glove.txt") 
# https://www.kaggle.com/watts2/glove6b50dtxt
glove_dimensions = 50
!python -m gensim.scripts.glove2word2vec --input  glove.6B.50d.txt --output glove.6B.50d.w2vformat.txt
word_vecs = KeyedVectors.load_word2vec_format("glove.6B.50d.w2vformat.txt")

/Users/bettyvandongen/opt/anaconda3/lib/python3.8/runpy.py:127: RuntimeWarning: 'gensim.scripts.glove2word2vec' found in sys.modules after import of package 'gensim.scripts', but prior to execution of 'gensim.scripts.glove2word2vec'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-06-16 19:55:38,335 - glove2word2vec - INFO - running /Users/bettyvandongen/opt/anaconda3/lib/python3.8/site-packages/gensim/scripts/glove2word2vec.py --input glove.6B.50d.txt --output glove.6B.50d.w2vformat.txt
2021-06-16 19:55:38,506 - glove2word2vec - INFO - converting 400000 vectors from glove.6B.50d.txt to glove.6B.50d.w2vformat.txt
2021-06-16 19:55:38,913 - glove2word2vec - INFO - Converted model with 400000 vectors and 50 dimensions


In [763]:
# Test word_vecs
word_vecs.word_vec('the')

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [764]:
# np.array(word_vecs.word_vec('the')).astype(np.float16)

In [765]:
num_tokens = len(voc) + 2
embedding_dim = glove_dimensions
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
# for i, word in enumerate(words_train):
#     embedding_vector = embeddings_index.get(word)
#     print(i)
    try:
        embedding_vector = word_vecs.word_vec(word)
#     if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
#     else:
    except KeyError:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 18285 words (3714 misses)


In [766]:
# words_test = list(set(df_test["word"].values))
# words_test.append("ENDPAD")

In [767]:
# n_words = len(words_train)
# n_words

In [768]:
# df_train[df_train['tag'].isna()]

In [769]:
from math import nan

tags = []
for index, tag in enumerate(set(df_train["tag"].values)):
    if tag is nan or isinstance(tag, float):
        tags.append('unk')
    else:
        tags.append(tag)
print(tags)

['CONTENT', 'SOURCE', 'O', 'CUE']


In [770]:
n_tags = len(tags); n_tags

4

In [771]:
from future.utils import iteritems
# word2idx_train = {w: i for i, w in enumerate(words_train)}
# word2idx_test = {w: i for i, w in enumerate(words_test)}

tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}

In [772]:
# word2idx_train['the']

In [773]:
# word2idx

In [774]:
tag2idx["O"]

2

In [775]:
print(tag2idx)

{'CONTENT': 0, 'SOURCE': 1, 'O': 2, 'CUE': 3}


In [776]:
idx2tag[3]

'CUE'

In [777]:
idx2tag

{0: 'CONTENT', 1: 'SOURCE', 2: 'O', 3: 'CUE'}

In [778]:
X_train = vectorizer(np.array([[s] for s in sentences_texts_train])).numpy()
X_test = vectorizer(np.array([[s] for s in sentences_texts_test])).numpy()
X_val = vectorizer(np.array([[s] for s in sentences_texts_val])).numpy()

In [779]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# X_train = [[word2idx_train[w[0]] for w in s] for s in sentences_train]
# X_test = [[word2idx_test[w[0]] for w in s] for s in sentences_test]

In [780]:
print(X_train[10])

[ 1055     9   755    10    71     5   406 18235  7390    15   439   227
   451     3     5  5392  3335    14     9    37 18916     7    86   101
     9    70  4031     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [781]:
np.array(X_train).shape

(30343, 220)

In [782]:
# X_train = pad_sequences(maxlen=maxlen, sequences=X_train, padding="post",value=n_words - 1)
# print(X_train[10])

In [783]:
# X_test = pad_sequences(maxlen=maxlen, sequences=X_test, padding="post",value=n_words - 1)
# print(X_test[10])

In [784]:
y_idx_train = [[tag2idx[w[1]] for w in s] for s in sentences_train]
# print('sentences_train[25]')
# print(sentences_train[25])
# print('y_idx_train[25]')
# print(y_idx_train[25])
# print('sentences_train[10]')
# print(sentences_train[10])
# print('y_idx_train[10]')
# print(y_idx_train[10])
# print(len(y_idx_train[10]))

In [785]:
y_idx_test = [[tag2idx[w[1]] for w in s] for s in sentences_test]
# print('sentences_test[25]')
# print(sentences_test[25])
# print('y_idx_test[25]')
# print(y_idx_test[25])
# print('sentences_test[10]')
# print(sentences_test[10])
# print('y_idx_test[10]')
# print(y_idx_test[10])
# print(len(y_idx_test[10]))

In [786]:
y_idx_val = [[tag2idx[w[1]] for w in s] for s in sentences_val]

In [787]:
y_train = pad_sequences(maxlen=maxlen, sequences=y_idx_train, padding="post", value=tag2idx["O"])
print(y_train[10])
print(len(y_train[10]))

[2 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
220


In [788]:
y_test = pad_sequences(maxlen=maxlen, sequences=y_idx_test, padding="post", value=tag2idx["O"])
print(y_test[10])
print(len(y_test[10]))

[1 1 1 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
220


In [789]:
y_val = pad_sequences(maxlen=maxlen, sequences=y_idx_val, padding="post", value=tag2idx["O"])
print(y_val[10])
print(len(y_val[10]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
220


In [790]:
from tensorflow.keras.utils import to_categorical
y_train = [to_categorical(i, num_classes=n_tags) for i in y_train]
print(y_train[10][0])

[0. 0. 1. 0.]


In [791]:
y_test = [to_categorical(i, num_classes=n_tags) for i in y_test]
print(y_test[10][0])

[0. 1. 0. 0.]


In [792]:
y_val = [to_categorical(i, num_classes=n_tags) for i in y_val]
print(y_val[10][0])

[1. 0. 0. 0.]


In [793]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

# X_train, y_train = X, y
# X_test, y_test = X, y

In [794]:
print(X_train[10][0])
print(X_test[10][0])
print(X_val[10][0])
print(y_train[10][0])
print(y_test[10][0])
print(y_val[10][0])

1055
2
47
[0. 0. 1. 0.]
[0. 1. 0. 0.]
[1. 0. 0. 0.]


In [795]:
from tensorflow.keras.models import Model, Sequential  # , Input
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import tensorflow.keras as k
import tensorflow as tf

In [796]:
print(k.__version__)

2.5.0


In [797]:
# import tensorflow.keras.utils as generic_utils
# from keras_contrib.layers.crf import CRF
# AttributeError: module 'tensorflow.compat.v2' has no attribute '__internal__'

In [798]:
word_embedding_size = embedding_dim  # glove_dimensions  # 300

https://github.com/xuxingya/tf2crf

More inspiration:
https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb
https://www.kaggle.com/nikkisharma536/ner-with-bilstm-and-crf

In [810]:
# Kaggle example code 
from tensorflow.keras.initializers import Constant

inputs = Input(shape=(None,), dtype="int64")  
# inputs = Input(shape=(maxlen,))
# https://stackoverflow.com/questions/55770009/how-to-use-a-pre-trained-embedding-matrix-in-tensorflow-2-0-rnn-as-initial-weigh
# https://keras.io/examples/nlp/pretrained_word_embeddings/ --> pretrained embeddings
# outputs = Embedding(input_dim=n_words, output_dim=word_embedding_size, input_length=maxlen)(inputs)
outputs = Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(embedding_matrix), trainable=False,)(inputs)
outputs = Bidirectional(LSTM(units=word_embedding_size, 
                             return_sequences=True, 
                             dropout=0.5,  # 0.2, 0.5, 
                             recurrent_dropout=0.5,  # 0.2, 0.5, 
                             kernel_initializer=k.initializers.he_normal()))(outputs)
# https://github.com/xuxingya/tf2crf: Add internal kernel like CRF in keras_contrib, so now there is no need to stack a Dense layer before the CRF layer.
# outputs = Dense(n_tags, activation="relu")(outputs)  # previously softmax output layer
outputs = TimeDistributed(Dense(n_tags, activation="relu"))(outputs)  # previously softmax output layer
# outputs = TimeDistributed(Dense(n_tags, activation=tensorflow.keras.activations.softmax))(outputs)  # previously softmax output layer

# crf = CRF(n_tags)  # CRF layer
# out = crf(outputs)  # output
# model = Model(input, out)

# adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)
adam = k.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999)


In [811]:
# Kaggle version (removed CRF code)

model = Model(inputs, outputs)

# https://stackoverflow.com/questions/61742556/valueerror-shapes-none-1-and-none-2-are-incompatible
# model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])  # categorical_accuracy?
# model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
# model.compile(optimizer=adam, loss=crf.loss_function, metrics=[crf.accuracy, 'accuracy'])# Saving the best only

In [812]:
# # Saving the best only
from tensorflow.keras.callbacks import ModelCheckpoint

# filepath="ner-bi-lstm-td-model-{val_accuracy:.2f}.hdf5"
filepath = 'tmp/checkpoint'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [813]:
# # tfcrf version
# model.fit(x=X_train, y=np.array(y_train), epochs=1, batch_size=2)
# model.save('tests/1')

In [814]:
# # Generate generalization metrics
# i = len(X_test) - 1 
# score = model.evaluate(np.array([X_test[:i]]), y_test, verbose=0)
# print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [815]:
# model.fit(X_train, np.array(y_train), batch_size=256, epochs=1, validation_split=0.1, verbose=1, callbacks=callbacks_list)
# model.fit(X_train, np.array(y_train), batch_size=256, epochs=50, validation_split=0.2, verbose=1, callbacks=callbacks_list)
model.fit(X_train, np.array(y_train), batch_size=256, epochs=50, validation_data=(X_val, y_val), verbose=1, callbacks=callbacks_list)

Epoch 1/50
 4/95 [>.............................] - ETA: 50s - loss: nan - accuracy: 0.0852

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
TP = {}
TN = {}
FP = {}
FN = {}
for tag in tag2idx.keys():
    TP[tag] = 0
    TN[tag] = 0    
    FP[tag] = 0    
    FN[tag] = 0    

def accumulate_score_by_tag(gt, pred):
    """
    For each tag keep stats
    """
    if gt == pred:
        TP[gt] += 1
    elif gt != 'O' and pred == 'O':
        FN[gt] +=1
    elif gt == 'O' and pred != 'O':
        FP[gt] += 1
    else:
        TN[gt] += 1

In [ ]:
print(X_test)

In [ ]:
### Predict single sentence

In [ ]:
# i = 2  # len(X_test) - 1  # Last one 
# p = model.predict(np.array([X_test[i]]))
# p = np.argmax(p, axis=-1)
# print(p.shape)
# gt = np.argmax(y_test[i], axis=-1)
# print(gt)
# print("{:14}: ({:5}): {}".format("Word", "True", "Pred"))
# print(p)
# for idx, (w,pred) in enumerate(zip(X_test[i],p[0])):
#     if words_test[w] == 'ENDPAD':
#         break
#     print("{:14}: ({:5}): {}".format(words_test[w],idx2tag[gt[idx]],tags[pred]))

In [ ]:
p = model.predict(np.array(X_test))

In [ ]:
p.shape

In [ ]:
# p

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
axis = 2

In [ ]:
np.argmax(p, axis=axis)[0]

In [ ]:
print(classification_report(np.argmax(y_test, axis=axis).ravel(), np.argmax(p, axis=axis).ravel(),labels=list(idx2tag.keys()), target_names=list(idx2tag.values())))

In [ ]:
for i, sentence in enumerate(X_test):
    y_hat = np.argmax(p[i], axis=-1)
    gt = np.argmax(y_test[i], axis=-1)
    for idx, (w,pred) in enumerate(zip(sentence,y_hat)):
        accumulate_score_by_tag(idx2tag[gt[idx]],tags[pred])

In [ ]:
for tag in tag2idx.keys():
    print(f'tag:{tag}')    
    print('\t TN:{:10}\tFP:{:10}'.format(TN[tag],FP[tag]))
    print('\t FN:{:10}\tTP:{:10}'.format(FN[tag],TP[tag]))  